# Ajout de nouvelles features

Ce fichier ajoute des features aux dataset d'origine telles que : 
- Le PIB du pays
- Le prix moyen par m2 dans chaque ville
- Le nombre de touristes par an dans chaque ville
- Le nombre d'habitants par km2 dans chaque ville.

In [1]:
# librairies 
#----------------------------------------------
#----- Calculs 
import pandas as pd
import numpy as np
from numpy import mean, std
import numpy.linalg as npl
import math 
from math import sqrt, log
import re #to find integers & floats in a string 
from collections import Counter #to count values in a dataframe or array
#----- Date
from datetime import date, timedelta, datetime #for the date
import datetime as dt
#save model 
import pickle 

import warnings
warnings.filterwarnings("ignore")

In [23]:
PATH = '.' #modifier si besoin le chemin du fichier

### Récupération & mise en forme des datasets

In [13]:
# Lecture du fichier des données tests
#----------------------------------------------

# récupération du fichier contentant l'ensemble des données issues des requêtes
pricing_requests = pd.read_csv(PATH + '//pricing_requests_clean.csv')

# récupération du fichier contenant les features des hotels
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])

print("pricing_requests.shape: ",pricing_requests.shape)
pricing_requests.head()

pricing_requests.shape:  (536088, 10)


,hotel_id,price,stock,city,date,language,mobile,avatar_id,request_number,request_nb
0,105,144,102,amsterdam,39,austrian,1,6056,1.0,1.0
1,880,113,81,amsterdam,39,austrian,1,6056,1.0,1.0
2,588,172,108,amsterdam,39,austrian,1,6056,1.0,1.0
3,352,105,84,amsterdam,39,austrian,1,6056,1.0,1.0
4,925,103,83,amsterdam,39,austrian,1,6056,1.0,1.0


In [14]:
# Obtenir des features en plus correspondant aux features de l'hotel : on fait une jointure 
#----------------------------------------------
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,request_number,request_nb,group,brand,parking,pool,children_policy
0,105,144,102,amsterdam,39,austrian,1,6056,1.0,1.0,Yin Yang,Ardisson,0,0,0
1,880,113,81,amsterdam,39,austrian,1,6056,1.0,1.0,Boss Western,Boss Western,0,0,0
2,588,172,108,amsterdam,39,austrian,1,6056,1.0,1.0,Independant,Independant,0,0,0
3,352,105,84,amsterdam,39,austrian,1,6056,1.0,1.0,Independant,Independant,1,0,0
4,925,103,83,amsterdam,39,austrian,1,6056,1.0,1.0,Boss Western,Boss Western,1,0,0


In [15]:
# Assigner le bon type aux variables
#----------------------------------------------
#--- Convert to integer: 
int_list = ["date","avatar_id","hotel_id","stock","request_number","price"] 
pricing_requests[int_list] = pricing_requests[int_list].astype(int) 
#--- Convert to categorical: 
pricing_requests["city"] = pd.Categorical(pricing_requests["city"],ordered=False)
pricing_requests["language"] = pd.Categorical(pricing_requests["language"],ordered=False)
pricing_requests["mobile"] = pd.Categorical(pricing_requests["mobile"],ordered=False)
pricing_requests["parking"] = pd.Categorical(pricing_requests["parking"],ordered=False)
pricing_requests["pool"] = pd.Categorical(pricing_requests["pool"],ordered=False)
pricing_requests["children_policy"] = pd.Categorical(pricing_requests["children_policy"],ordered=False)
pricing_requests["group"] = pd.Categorical(pricing_requests["group"],ordered=False)
pricing_requests["brand"] = pd.Categorical(pricing_requests["brand"],ordered=False)

In [16]:
# Vérification du type des variables
#----------------------------------------------
pricing_requests.dtypes

hotel_id              int32
price                 int32
stock                 int32
city               category
date                  int32
language           category
mobile             category
avatar_id             int32
request_number        int32
request_nb          float64
group              category
brand              category
parking            category
pool               category
children_policy    category
dtype: object

### ajout du PIB par pays
**source:** https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal) estimate by IMF consulté le 17/11/2022.

- amsterdam: netherlands
- copenhagen: denmark
- paris: france
- sofia: bulgaria 
- vienna: austria
- rome:  italy
- madrid: spain
- vilnius: lithuania
- valletta: malta

In [17]:
# Ajout du PIB par pays
#-----------------------------------------
# dictionnaire contenant la ville et le pib du pays où se situe la ville
PIB = {"amsterdam": 990583,
     "copenhagen": 386724,
     "paris": 2778090,
     "sofia": 85008,
     "vienna": 468046,
     "rome": 1996934,
     "madrid":1389927,
     "vilnius":68031,
     "valletta":17156}

# ajout du pib au dataframe général 
pricing_requests["pib"] = np.zeros(len(pricing_requests))

for city in PIB.keys():
    city_idx = np.where(pricing_requests["city"]== city)
    pricing_requests["pib"].iloc[city_idx] = PIB[city]

### ajout du prix moyen par m2 dans chaque ville:

**source**: https://checkinprice.com/europe-square-meter-prices/ consulté le 17/11/22. Données datant de 2018.

In [18]:
# Ajout du prix moyen par m2 par ville
#-----------------------------------------
# dictionnaire contenant la ville et le prix moyen par m2 en euros
price_m2 = {"amsterdam": 4610,
            "copenhagen": 5236,
            "paris": 9160,
            "sofia": 1095,
            "vienna": 6550,
            "rome": 3044,
            "madrid": 3540,
            "vilnius": 1469,
            "valletta": 3600}

# ajout du prix par m2 au dataframe général 
pricing_requests["price_m2"] = np.zeros(len(pricing_requests))

for city in price_m2.keys():
    city_idx = np.where(pricing_requests["city"]== city)
    pricing_requests["price_m2"].iloc[city_idx] = price_m2[city]

### ajout du nombre de touristes par an dans chaque ville

**source:** https://www.indexmundi.com/facts/indicators/ST.INT.ARVL/rankings consulté 17/11/22. Chiffres de 2020. 

In [19]:
# Ajout du nombre de touristes par pays en 2020
#-----------------------------------------
# dictionnaire contenant le nombre de touristes par pays en 2020
nb_tourists = {"amsterdam": 7265, 
                "copenhagen": 15595 ,
                "paris": 117109,
                "sofia": 4973,
                "vienna": 15091,
                "rome": 38419,
                "madrid": 36410,
                "vilnius": 2284,
                "valletta": 718}
# ajouter 10^3 à toutes les valeurs pour obtenir les vraies valeurs

# ajout du nombre de touristes par pays
pricing_requests["nb_tourists"] = np.zeros(len(pricing_requests))

for city in nb_tourists.keys():
    city_idx = np.where(pricing_requests["city"]== city)
    pricing_requests["nb_tourists"].iloc[city_idx] = nb_tourists[city]

### ajout du nombre d'habitants par km2 dans chaque ville.

**source**: wikikepia: https://fr.wikipedia.org/wiki/La_Valette, https://fr.wikipedia.org/wiki/Vilnius, https://fr.wikipedia.org/wiki/Madrid, https://fr.wikipedia.org/wiki/Rome, https://fr.wikipedia.org/wiki/Vienne_(Autriche), https://fr.wikipedia.org/wiki/Sofia, https://fr.wikipedia.org/wiki/Paris, https://fr.wikipedia.org/wiki/Copenhague, https://fr.wikipedia.org/wiki/Amsterda

In [20]:
# Ajout du nombre de d'habitants par km2 dans chaque ville
#-----------------------------------------
# dictionnaire contenant le nombre de touristes par pays en 2020
nb_hab_km2 = {"amsterdam": 3530, 
            "copenhagen": 7064,
            "paris": 20545,
            "sofia": 7354,
            "vienna": 4607,
            "rome": 2213,
            "madrid": 5437,
            "vilnius": 1432,
            "valletta": 8344}

# ajout du nombre d'habitants par km2 par ville
pricing_requests["nb_hab_km2"] = np.zeros(len(pricing_requests))

for city in nb_hab_km2.keys():
    city_idx = np.where(pricing_requests["city"]== city)
    pricing_requests["nb_hab_km2"].iloc[city_idx] = nb_hab_km2[city]

### Enregistrement du nouveau dataset

In [22]:
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,request_number,request_nb,group,brand,parking,pool,children_policy,pib,price_m2,nb_tourists,nb_hab_km2
0,105,144,102,amsterdam,39,austrian,1,6056,1,1.0,Yin Yang,Ardisson,0,0,0,990583.0,4610.0,7265.0,3530.0
1,880,113,81,amsterdam,39,austrian,1,6056,1,1.0,Boss Western,Boss Western,0,0,0,990583.0,4610.0,7265.0,3530.0
2,588,172,108,amsterdam,39,austrian,1,6056,1,1.0,Independant,Independant,0,0,0,990583.0,4610.0,7265.0,3530.0
3,352,105,84,amsterdam,39,austrian,1,6056,1,1.0,Independant,Independant,1,0,0,990583.0,4610.0,7265.0,3530.0
4,925,103,83,amsterdam,39,austrian,1,6056,1,1.0,Boss Western,Boss Western,1,0,0,990583.0,4610.0,7265.0,3530.0


In [24]:
pricing_requests.to_csv(PATH + '//pricing_requests_clean_addedFeatures.csv')